### The Stanford Data Project Analysis - TX
On a typical day in the United States, police officers make more than 50,000 traffic stops. Our team is gathering, analyzing, and releasing records from millions of traffic stops by law enforcement agencies across the country. Our goal is to help researchers, journalists, and policymakers investigate and improve interactions between police and the public.

In [ ]:
import pandas as pd
import requests, zipfile, io

### Data aquisition
The data of the project is in a form of compressed file hosted online, it will be downloaded and extracted to the project directory. You can always host the data anywhere else and change the pointer in the read csv  line

In [ ]:
url="https://stacks.stanford.edu/file/druid:hp256wp2687/hp256wp2687_tx_houston_2019_08_13.csv.zip"

In [3]:
r = requests.get(url)
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall()

In [ ]:
df=pd.read_csv('tx_houston_2019_08_13.csv')

### Data Exploration
In this section we will perform some basic data analysis and exploration tasks in order to understand the data. worth noting also that data comes with shape file that will help in data spatial distrubution.